# Graph Coloring Problem
#### In this classic graph theory problem, a simple, undirected graph is given. How can we assign a color to each vertex such that no two adjacent (connected) vertices have the same color? The smallest such number is called the chromatic number. For planar graphs, we know (because of the Four Color Theorem) that an upper bound for the chromatic number is 4. In general, though, the chromatic number may be much higher that, especially if the graph is dense. The filenames are given in the form "gc_v_d" where v is the number of vertices and d is the density of the graph. So the graph in file "gc_100_1" has 100 vertices but is not very dense. The graph in file "gc_1000_9" has 1000 vertices and is very dense. In general, it has harder to to find the chromatic number for more dense graphs.

#### Below, I use networkx to get very quick approximations of the chromatic number (although some may be much higher than the actual chromatic number). I then use OR-Tools to find the optimal answer (aka the actual chromatic number). However, this is a slow process (especially for larger or more dense graphs). 

More information on this problem can be found here:
https://en.wikipedia.org/wiki/Graph_coloring

In [59]:
import constraint
import numpy as np

import matplotlib.pyplot as plt
import networkx as nx

import time

# Using NetworkX
Find approximations of answers

In [60]:
def greedy_networkx(filename):
    f = open(filename, 'r')
    input_data = f.read()

    lines = input_data.split('\n')

    #get data
    firstLine = lines[0].split()
    node_count = int(firstLine[0])
    edge_count = int(firstLine[1])

    #create graph from data
    G=nx.Graph()
    for i in range(1, edge_count + 1):
        node_1 = int(lines[i].split()[0])
        node_2 = int(lines[i].split()[1])
        G.add_edges_from([(node_1, node_2)])


    solution = [i[1] for i in sorted(nx.greedy_color(G, strategy='DSATUR', interchange=False).items())]  

    minimum_coloring = len(set(solution))

    return minimum_coloring, solution




In [61]:
filename = 'data/gc_50_3'

start = time.time()
min_coloring, solution = greedy_networkx(filename)
print(min_coloring)
print()
print(time.time()-start, 'seconds')


7

0.005982160568237305 seconds


In [62]:
filename = 'data/gc_100_1'

start = time.time()
min_coloring, solution = greedy_networkx(filename)
print(min_coloring)
print()
print(time.time()-start, 'seconds')

6

0.013608217239379883 seconds


In [63]:
filename = 'data/gc_100_3'

start = time.time()
min_coloring, solution = greedy_networkx(filename)
print(min_coloring)
print()
print(time.time()-start, 'seconds')

12

0.016910076141357422 seconds


In [64]:
filename = 'data/gc_1000_9'

start = time.time()
min_coloring, solution = greedy_networkx(filename)
print(min_coloring)
print()
print(time.time()-start, 'seconds')

297

2.378542184829712 seconds


# Using OR-Tools
Constraint Programming. Optimal answer. Very slow for large and/or dense graphs

In [65]:
from ortools.constraint_solver import pywrapcp

In [66]:

def graph_coloring(filename):
    f = open(filename, 'r')
    input_data = f.read()

    lines = input_data.split('\n')

    #get data
    firstLine = lines[0].split()
    node_count = int(firstLine[0])
    edge_count = int(firstLine[1])

    #vertices
    V = range(node_count)

    #edges
    E = []
    for i in range(1, edge_count + 1):
        node_1 = int(lines[i].split()[0])
        node_2 = int(lines[i].split()[1])
        E.append([node_1, node_2])

    #find upper bound using networkx greedy algorithm
    max_num_colors, upper_bound_solution = greedy_networkx(filename)


    solver = pywrapcp.Solver('Map coloring')

    # decision variables
    x = [solver.IntVar(1, max_num_colors, 'x[%i]' % i) for i in V]

    # number of colors used, to minimize
    max_color = solver.Max(x).Var()

    # constraints

    # adjacent nodes cannot be assigned the same color
    # (and adjust to 0-based)
    for i in range(edge_count):
        solver.Add(x[E[i][0]] != x[E[i][1]])

    # symmetry breaking
    for i in range(max_num_colors):
        solver.Add(x[i] <= i+1);

    # objective (minimize the number of colors)
    objective = solver.Minimize(max_color, 1)


    # solution
    solution = solver.Assignment()
    solution.Add(x)
    solution.Add(max_color)

    db = solver.Phase(x,
                    # solver.CHOOSE_FIRST_UNBOUND,
                    solver.CHOOSE_MIN_SIZE_LOWEST_MAX,

                    # solver.ASSIGN_MIN_VALUE
                    solver.ASSIGN_MIN_VALUE
                    )

    solver.NewSearch(db, [objective])
    num_solutions = 0
    while solver.NextSolution():
        num_solutions += 1
#         print("x:", [int(x[i].Value()) for i in V])
#         print("max_color:", max_color.Value())
#         print()
        solution = [int(x[i].Value()) for i in V]
        min_colors = max_color.Value()
    solver.EndSearch()

    print('minimum colors required:', min_colors)
    print('possible solution:', solution)


    #return max_color.Value(), [int(x[i].Value()) for i in V]

In [67]:
filename = 'data/gc_50_3'

start = time.time()
graph_coloring(filename)
print()
print(time.time()-start, 'seconds')

minimum colors required: 6
possible solution: [1, 1, 1, 2, 1, 3, 2, 5, 3, 6, 3, 5, 3, 4, 2, 3, 5, 4, 6, 1, 6, 6, 3, 3, 3, 4, 3, 4, 4, 6, 6, 2, 1, 1, 6, 3, 6, 1, 4, 4, 4, 6, 5, 5, 2, 5, 6, 2, 6, 4]

0.037158966064453125 seconds


In [68]:
filename = 'data/gc_100_1'

start = time.time()
graph_coloring(filename)
print()
print(time.time()-start, 'seconds')

minimum colors required: 5
possible solution: [1, 1, 1, 2, 2, 3, 3, 2, 2, 1, 1, 2, 2, 4, 3, 2, 3, 2, 4, 2, 3, 1, 3, 3, 2, 1, 1, 5, 3, 1, 1, 2, 2, 1, 4, 5, 3, 3, 1, 3, 4, 4, 3, 1, 2, 4, 5, 2, 4, 5, 3, 2, 3, 1, 1, 2, 1, 4, 4, 5, 2, 4, 2, 3, 4, 3, 2, 2, 4, 4, 5, 3, 5, 1, 5, 1, 5, 5, 1, 5, 3, 4, 5, 4, 1, 4, 2, 3, 1, 3, 4, 3, 3, 3, 5, 1, 1, 1, 5, 1]

0.22486329078674316 seconds
